<a href="https://colab.research.google.com/github/Praz98/project/blob/main/Item_Demand_Forecasting_project_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

#Importing all the necessary packages 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sb 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import LabelEncoder, StandardScaler 
from sklearn import metrics 
from sklearn.svm import SVC 
from xgboost import XGBRegressor 
from sklearn.ensemble import RandomForestRegressor 
from sklearn.metrics import mean_absolute_error as mae 
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score,r2_score


import warnings 
warnings.filterwarnings('ignore')
 

In [ ]:
df  = pd.read_csv('/content/drive/MyDrive/train 2.csv')
display(df.head()) 
display(df.tail())

In [ ]:
columns=list(df.columns)
display(columns)
display(df.shape)


In [ ]:

df.info()

In [ ]:
df.describe()


In [ ]:
print(df.isnull().sum())

print(df.isna().sum())

In [ ]:
df.dtypes

In [ ]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
df.dtypes

In [ ]:
print(df["store"].nunique())
print(df["item"].nunique())

In [ ]:
df.groupby(["store"])["item"].nunique()

In [ ]:

df.groupby(["item", "date"],as_index=False)['sales'] .agg({"sales": "sum"})

In [ ]:
from datetime import timedelta
df['start_date']=df['date']
df['end_date']=df['date'] + timedelta(90)

In [ ]:
df.head(5)

In [ ]:
df1=pd.DataFrame(df)

In [ ]:
df1

In [ ]:
x=pd.DataFrame(df.set_index('date').sort_index(ascending=False).groupby(['item'],as_index=False).rolling('90d').sales.sum().sort_index().to_frame()).reset_index()

In [ ]:
x.head(10)

In [ ]:
df=x.copy()

In [ ]:
df.head()

In [ ]:
def create_date_time_features(df):
    """
    Create time series features based on time series index.
    """
    df = df.copy()
    df['month'] = df.date.dt.month
    df['day_of_month'] = df.date.dt.day
    df['day_of_year'] = df.date.dt.dayofyear
    df['week_of_year'] = df.date.dt.weekofyear
    df['day_of_week'] = df.date.dt.weekday
    df['year'] = df.date.dt.year
    df['quarter'] = df.date.dt.quarter
    df["is_wknd"] = (df.date.dt.weekday > 4).astype(int)
    df['is_month_start'] = df.date.dt.is_month_start.astype(int)
    df['is_month_end'] = df.date.dt.is_month_end.astype(int)
    # 0: Winter - 1: Spring - 2: Summer - 3: Fall
    df["season"] = np.where(df.month.isin([12,1,2]), 0, 1)
    df["season"] = np.where(df.month.isin([6,7,8]), 2, df["season"])
    df["season"] = np.where(df.month.isin([9, 10, 11]), 3, df["season"])
    return df
df= create_date_time_features(df)

In [ ]:
df

In [ ]:

features = ['year', 'month','week_of_year','quarter', 'season'] 
plt.subplots(figsize=(20, 10)) 
for i, col in enumerate(features):
  plt.subplot(2, 3, i + 1) 
  df.groupby(col).mean()['sales'].plot.bar() 
plt.show() 

In [ ]:

plt.figure(figsize=(10,5)) 
df.groupby('day_of_month').mean()['sales'].plot() 
plt.show()
 

In [ ]:

plt.subplots(figsize=(12, 5)) 
plt.subplot(1, 2, 1) 
sb.distplot(df['sales']) 
plt.subplot(1, 2, 2) 
sb.boxplot(df['sales']) 
plt.show() 

In [ ]:


plt.figure(figsize=(10, 10)) 
sb.heatmap(df.corr() > 0.8, 

           annot=True, 

           cbar=False) 
plt.show() 
 

In [ ]:
def item_select(itm):
  df_item=df[df['item']==itm]
  df_item=df.sample(frac=1)
  # Train set until the beginning of 2017 (until the end of 2016).
  train = df.loc[(df["date"] < "2017-01-01"), :]
  # First 3 months of 2017 validation set.
  test = df.loc[(df["date"] >= "2017-01-01") & (df["date"] < "2017-04-01"), :]

  cols = [col for col in train.columns if col not in ['date', 'id', "sales", "year"]]

  Y_train = train['sales']
  X_train = train[cols]

  Y_test = test['sales']
  X_test = test[cols]

  scaler=StandardScaler()
  X_train=scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)
  return(X_train,Y_train,X_test,Y_test)


In [ ]:
def lin_model(itm):
  x_train,y_train,x_test,y_test=item_select(itm)
  regressor=LinearRegression()
  regressor.fit(x_train,y_train)
  y_pred=regressor.predict(X_test)
  return(regressor.score(x_test,y_test))


def knn_model(itm):
  x_train,y_train,x_test,y_test=item_select(itm)
  knn=KNeighborsRegressor(hp_val['k_value'].loc[hp_val,item[itm]])
  knn.fit(x_train,y_train)
  return(knn.score(x_test,y_test))

def dt_score(itm):
  x_train,y_train,x_test,y_test=item_select(itm)
  dt=DecisionTreeRegressor(max_depth=hp_val['Depth'].loc[hp_val.Item[itm]])
  dt.fit(x_train,y_train)
  y_pred=dt.predict(X_test)
  dt_r2_score=r2_score(y_test,y_pred)
  return(dt_r2_score)

def decision_tree(itm):
  x_train,y_train,x_test,y_test=item_select(itm)
  dt=DecisionTreeRegressor(max_depth=hp_val['Depth'].loc[hp_val.Item[itm]])
  dt.fit(x_train,y_train)
  return(np.mean(cross_val_score(dt,x_train,y_train,cv=10)))


In [ ]:
#hp for knn
k_itr,item = 0,0
k_ref,itm = [],[]
for j in range(1,51):
  cv_max=0
  X_train, Y_train, X_test, Y_test =item_select(j)
  for i in[1,2,3,4,5,6,7,8,9,10,20,50]:
     knn=KNeighborsRegressor(i)
     knn.fit(X_train,Y_train)
     cv=np.mean(cross_val_score(knn,X_train,Y_train,cv=10))
     if (cv_max<cv):
       cv_max=cv 
       item =j
       K_itr=i
  k_ref.append(k_itr)
  itm.append(item)

hp_val=pd.DataFrame({'Item':itm,'k_value':k_ref},index=itm)


In [ ]:
#hp for dt
d_itr,item = 0,0
d_ref,itm = [],[]
for j in range(1,51):
  dep_max=0
  X_train, Y_train, X_test, Y_test =item_select(j)
  for i in[1,2,3,4,5,6,7,8,9,10,20,21,22,23,24,25,30,40,60]:
    dt=DecisionTreeRegressor(max_depth=depth)
    dt.fit(X_train,Y_train) 
    valAccuracy= cross_val_score(dt,X_train,Y_train,cv=10)
    dep=np.mean(valAccuracy)
    if(dep_max < dep):
      dep_max = dep
      d_itr = depth
  d_ref.append(d_itr)

hp_val['Depth']=d_ref




In [ ]:
cols = [col for col in df.columns if col not in ['date', 'id', "sales", "year"]]

In [ ]:
itm=1
X_train = item_select(j)
Y_train = item_select(j)
X_test = item_select(j)
Y_test = item_select(j)
dt=DecisionTreeRegressor(max_depth=hp_val['Depth'].loc[hp_val.Item[itm]])
dt.fit(X_train,Y_train)
importance = dt.feature_importances_
list(zip(importance,cols))


In [ ]:
k_temp,lin_temp,dtree_temp,dtree_r2_score,temp = [],[],[],[],[]
for i in range(1,51):
  k_temp.append(knn_model(i))
  lin_temp.append(lin_model(i))
  dtree_temp.append(decision_tree(i))
  dtree_r2_score.append(dt_score(i))
  temp.append(i)
df_ml_score=pd.DataFrame({'item':temp,'dtree_cv_score':dtree_temp,'dt_r2':dtree_r2_score,'knn_score':k_temp,'lin_R2score':lin_temp})